# PoSME Reference Benchmark
**Proof of Sequential Memory Execution** -- draft-condrey-cfrg-posme

This notebook compiles and runs the PoSME benchmark on Colab hardware.
No GPU required (CPU benchmark). Click **Runtime > Run all** to execute.

The benchmark validates two machine-independent claims:
1. Hash fraction < 10% at production arena sizes
2. Step time increases with arena size (cache-to-DRAM transition)


In [ ]:
# Install Rust (~30 seconds)
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --quiet 2>/dev/null
import os
os.environ['PATH'] = os.path.expanduser('~/.cargo/bin') + ':' + os.environ['PATH']
!rustc --version

In [ ]:
# Write benchmark source (embedded, no network dependency)
source = "// PoSME Reference Benchmark\n// =========================\n// Proof of Sequential Memory Execution -- single-file benchmark.\n//\n// Validates the latency-bound ASIC resistance claim by measuring\n// the per-step cost breakdown (hash vs memory) across arena sizes.\n//\n// BUILD:  rustc -O posme-bench.rs -o posme-bench\n// RUN:    ./posme-bench\n//\n// No external dependencies. Uses a built-in minimal hash (SipHash\n// via std) for portability. Production implementations should use\n// BLAKE3; the latency-bound property is independent of hash choice\n// as long as hash time << memory read time.\n\nuse std::collections::hash_map::DefaultHasher;\nuse std::hash::{Hash, Hasher};\nuse std::time::Instant;\n\nconst LAMBDA: usize = 32;\n\n#[derive(Clone, Copy)]\nstruct Block {\n    data: [u8; LAMBDA],\n    causal: [u8; LAMBDA],\n}\n\nimpl Block {\n    fn zeroed() -> Self {\n        Self {\n            data: [0u8; LAMBDA],\n            causal: [0u8; LAMBDA],\n        }\n    }\n}\n\nfn fast_hash(inputs: &[&[u8]]) -> [u8; LAMBDA] {\n    let mut h = DefaultHasher::new();\n    for inp in inputs {\n        inp.hash(&mut h);\n    }\n    let v = h.finish();\n    let mut out = [0u8; LAMBDA];\n    let bytes = v.to_le_bytes();\n    // Fill 32 bytes by hashing twice with different seeds\n    out[..8].copy_from_slice(&bytes);\n    let mut h2 = DefaultHasher::new();\n    v.hash(&mut h2);\n    let v2 = h2.finish();\n    out[8..16].copy_from_slice(&v2.to_le_bytes());\n    let mut h3 = DefaultHasher::new();\n    v2.hash(&mut h3);\n    let v3 = h3.finish();\n    out[16..24].copy_from_slice(&v3.to_le_bytes());\n    let mut h4 = DefaultHasher::new();\n    v3.hash(&mut h4);\n    out[24..32].copy_from_slice(&h4.finish().to_le_bytes());\n    out\n}\n\nfn i2osp(x: u32) -> [u8; 4] {\n    x.to_be_bytes()\n}\n\nfn addr_from(cursor: &[u8; LAMBDA], j: u32, n: usize) -> usize {\n    let h = fast_hash(&[b\"addr\", cursor, &j.to_be_bytes()]);\n    u32::from_be_bytes([h[0], h[1], h[2], h[3]]) as usize % n\n}\n\nfn initialize(blocks: &mut [Block], seed: &[u8]) {\n    let n = blocks.len();\n    blocks[0].data = fast_hash(&[b\"init\", seed, &i2osp(0)]);\n    blocks[0].causal = fast_hash(&[b\"causal\", seed, &i2osp(0)]);\n    for i in 1..n {\n        let prev = blocks[i - 1].data;\n        let skip = blocks[i / 2].data;\n        blocks[i].data = fast_hash(&[b\"init\", seed, &i2osp(i as u32), &prev, &skip]);\n        blocks[i].causal = fast_hash(&[b\"causal\", seed, &i2osp(i as u32)]);\n    }\n}\n\n#[inline(never)]\nfn posme_step(blocks: &mut [Block], t_prev: &[u8; LAMBDA], t: u32, d: usize) -> [u8; LAMBDA] {\n    let n = blocks.len();\n    let mut cursor = *t_prev;\n\n    // THE CRITICAL LOOP: pointer-chase with data-dependent addressing.\n    // cursor = H(cursor || block.data || block.causal)\n    // Each iteration MUST wait for the memory read before computing\n    // the next address. This is where latency-hardness lives.\n    for j in 0..d {\n        let a = addr_from(&cursor, j as u32, n);\n        let val = blocks[a];\n        cursor = fast_hash(&[&cursor, &val.data, &val.causal]);\n    }\n\n    // Symbiotic write\n    let w = addr_from(&cursor, d as u32, n);\n    let old = blocks[w];\n    blocks[w].data = fast_hash(&[&old.data, &cursor, &old.causal]);\n    blocks[w].causal = fast_hash(&[&old.causal, &cursor, &i2osp(t)]);\n\n    // Transcript chain\n    fast_hash(&[t_prev, &i2osp(t), &cursor])\n}\n\nfn hex8(b: &[u8; LAMBDA]) -> String {\n    b[..8].iter().map(|x| format!(\"{:02x}\", x)).collect()\n}\n\nfn format_size(bytes: usize) -> String {\n    if bytes >= 1 << 30 {\n        format!(\"{:.1} GiB\", bytes as f64 / (1u64 << 30) as f64)\n    } else if bytes >= 1 << 20 {\n        format!(\"{:.0} MiB\", bytes as f64 / (1u64 << 20) as f64)\n    } else {\n        format!(\"{:.0} KiB\", bytes as f64 / (1u64 << 10) as f64)\n    }\n}\n\nfn run_benchmark(n: usize, k: u32, d: usize, label: &str) {\n    let arena_bytes = n * 2 * LAMBDA;\n    let rho = k as f64 / n as f64;\n\n    println!(\"  {:.<44} {} arena, {} steps, rho={:.1}\",\n        format!(\"{} \", label), format_size(arena_bytes), k, rho);\n\n    let mut blocks = vec![Block::zeroed(); n];\n    initialize(&mut blocks, b\"posme-bench-v1\");\n\n    let mut transcript = fast_hash(&[b\"transcript\", b\"posme-bench-v1\"]);\n\n    let start = Instant::now();\n    for t in 1..=k {\n        transcript = posme_step(&mut blocks, &transcript, t, d);\n    }\n    let elapsed = start.elapsed();\n\n    let total_s = elapsed.as_secs_f64();\n    let avg_ns = total_s / k as f64 * 1e9;\n    let hash_est_ns = (d as f64 + 3.0) * 5.0; // ~5ns per hash (SipHash)\n    let _mem_ns = avg_ns - hash_est_ns;\n    let hash_pct = hash_est_ns / avg_ns * 100.0;\n\n    println!(\"    Total time:        {:.3}s\", total_s);\n    println!(\"    Avg step:          {:.0} ns\", avg_ns);\n    println!(\"    Hash fraction:     {:.1}%\", hash_pct);\n    println!(\"    T_K:               {}...\", hex8(&transcript));\n    println!(\"    TMTO (alpha=0):    {:.1}x\", 1.0 + (2.0 * rho + 1.0));\n    println!();\n}\n\nfn main() {\n    let d: usize = 8;\n\n    println!();\n    println!(\"  \u2554\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2557\");\n    println!(\"  \u2551         PoSME  Reference  Benchmark  v1.0              \u2551\");\n    println!(\"  \u2551   Proof of Sequential Memory Execution                 \u2551\");\n    println!(\"  \u2551   draft-condrey-cfrg-posme                             \u2551\");\n    println!(\"  \u2560\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2563\");\n    println!(\"  \u2551  Reads/step (d):  8                                    \u2551\");\n    println!(\"  \u2551  Hash:            SipHash (std, portable)              \u2551\");\n    println!(\"  \u2551  Block size:      64 bytes (32 data + 32 causal)       \u2551\");\n    println!(\"  \u255a\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u255d\");\n    println!();\n\n    println!(\"  \u2500\u2500 Phase 1: Cache Scaling \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\");\n    println!(\"  Measures per-step cost across arena sizes to demonstrate\");\n    println!(\"  the transition from cache-bound to memory-bound behavior.\");\n    println!();\n\n    // Small arenas (fit in cache) -- short runs\n    run_benchmark(1 << 14, 1 << 14, d, \"L2-resident (1 MiB)\");\n    run_benchmark(1 << 16, 1 << 16, d, \"L3-resident (4 MiB)\");\n    run_benchmark(1 << 18, 1 << 18, d, \"L3-edge (16 MiB)\");\n    run_benchmark(1 << 20, 1 << 20, d, \"DRAM-spill (64 MiB)\");\n\n    println!(\"  \u2500\u2500 Phase 2: Production Parameters \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\");\n    println!(\"  1 GiB arena, rho=1 (K=N). This is the minimum recommended\");\n    println!(\"  configuration for TMTO resistance.\");\n    println!();\n\n    let n_prod = 1 << 24; // 16M blocks = 1 GiB\n    let k_prod = n_prod as u32; // rho = 1\n    run_benchmark(n_prod, k_prod, d, \"Production (1 GiB, rho=1)\");\n\n    println!(\"  \u2500\u2500 Interpretation Guide \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\");\n    println!();\n    println!(\"  MACHINE-DEPENDENT (will vary by hardware):\");\n    println!(\"    - Absolute step time (ns)\");\n    println!(\"    - Total execution time (s)\");\n    println!();\n    println!(\"  MACHINE-INDEPENDENT (should hold on any hardware):\");\n    println!(\"    - Hash fraction < 10% at 1 GiB arena\");\n    println!(\"    - Step time INCREASES with arena size\");\n    println!(\"    - TMTO penalty matches formula: 1 + (1-a)(2p+1)\");\n    println!();\n    println!(\"  WHY THIS MATTERS:\");\n    println!(\"    If hash fraction is small, an ASIC that computes\");\n    println!(\"    hashes 100x faster gains almost nothing. The\");\n    println!(\"    bottleneck is memory latency, which is bounded by\");\n    println!(\"    physics (~2x DDR5 vs HBM3, ~35ns vs ~20ns).\");\n    println!();\n    println!(\"  For full details: draft-condrey-cfrg-posme\");\n    println!(\"  https://lf-decentralized-trust-labs.github.io/proof-of-effort/\");\n    println!();\n}\n"
with open('posme-bench.rs', 'w') as f:
    f.write(source)
print(f'Wrote {len(source)} bytes to posme-bench.rs')

In [ ]:
# Compile and run
!rustc -O posme-bench.rs -o posme-bench 2>&1
!./posme-bench

In [ ]:
# Report hardware info
!echo '=== Hardware ==='
!lscpu 2>/dev/null | grep -E 'Model name|CPU MHz|Cache|Architecture' || echo 'lscpu unavailable'
!echo ''
!echo '=== Memory ==='
!free -h 2>/dev/null | head -3 || echo 'free unavailable'
!echo ''
!echo '=== GPU (if present) ==='
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader 2>/dev/null || echo 'No GPU'